<a href="https://colab.research.google.com/github/qdzwsx/Chinese-LLaMA-Alpaca/blob/main/finetune_chinese_alpaca_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial on instruction tuning of Chinese-Alpaca-7B

More info: https://github.com/ymcui/Chinese-LLaMA-Alpaca

## Install Dependencies

In [ ]:
!pip install transformers==4.28.1
!pip install git+https://github.com/huggingface/peft.git@13e53fc
!pip install datasets
!pip install sentencepiece
!pip install deepspeed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git (to revision 13e53fc) to /tmp/pip-req-build-_8xtpwir
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-_8xtpwir
  Running command git checkout -q 13e53fc
  Resolved https://github.com/huggingface/peft.git to commit 13e53fc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 8.9 MB/s eta 0:00:00
  Created

## Clone our repository





In [ ]:
!git clone https://github.com/ymcui/Chinese-LLaMA-Alpaca.git

Cloning into 'Chinese-LLaMA-Alpaca'...
remote: Enumerating objects: 1230, done.
remote: Counting objects: 100% (422/422), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 1230 (delta 265), reused 338 (delta 243), pack-reused 808
Receiving objects: 100% (1230/1230), 22.47 MiB | 26.41 MiB/s, done.
Resolving deltas: 100% (727/727), done.


## Instruction tuning for Alpaca-7B

This follows the setting in https://github.com/ymcui/Chinese-LLaMA-Alpaca/wiki/SFT-Script, except that to simplify the tutorial,
- continue training the Chinese-Alpaca-LoRA
- only train 100 steps
- omit validation

In [ ]:
!mkdir Chinese-LLaMA-Alpaca/sft_data
!cp Chinese-LLaMA-Alpaca/data/alpaca_data_zh_51k.json Chinese-LLaMA-Alpaca/sft_data

In [ ]:
!cd Chinese-LLaMA-Alpaca/scripts && torchrun --nnodes 1 --nproc_per_node 1 run_clm_sft_with_peft.py \
    --deepspeed ds_zero2_no_offload.json \
    --model_name_or_path decapoda-research/llama-7b-hf \
    --tokenizer_name_or_path ziqingyang/chinese-alpaca-lora-7b \
    --dataset_dir /content/Chinese-LLaMA-Alpaca/sft_data \
    --validation_split_percentage 0.001 \
    --per_device_train_batch_size 1 \
    --do_train \
    --fp16 \
    --seed $RANDOM \
    --max_steps 100 \
    --lr_scheduler_type cosine \
    --learning_rate 1e-4 \
    --warmup_ratio 0.03 \
    --weight_decay 0 \
    --logging_strategy steps \
    --logging_steps 10 \
    --save_strategy steps \
    --save_total_limit 3 \
    --save_steps 50 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 8 \
    --max_seq_length 512 \
    --output_dir /content/output_model \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --torch_dtype float16 \
    --peft_path ziqingyang/chinese-alpaca-lora-7b \
    --gradient_checkpointing \
    --ddp_find_unused_parameters False

2023-06-02 01:36:19.600008: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-06-02 01:36:21,573] [INFO] [comm.py:622:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
06/02/2023 01:36:28 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True
[INFO|configuration_utils.py:668] 2023-06-02 01:36:28,850 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--decapoda-research--llama-7b-hf/snapshots/5f98eefcc80e437ef68d457ad7bf167c2c6a1348/config.json
[INFO|configuration_utils.py:720] 2023-06-02 01:36:28,851 >> Model config LlamaConfig {
  "_name_or_path": "decapoda-research/llama-7b-hf",
  "architectures": [
    "LLaMAForCausalLM"
  ],
  "bos_token_id": 0,
  "eos_token_id": 1,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "

After training, rename saved `pytorch_model.bin` to `adapter_model.bin`

In [ ]:
!mkdir output_model/peft_model
!mv output_model/pytorch_model.bin output_model/peft_model/adapter_model.bin

mkdir: cannot create directory ‘output_model/peft_model’: No such file or directory
mv: cannot stat 'output_model/pytorch_model.bin': No such file or directory


Lastly, you need to manually create an `adapter_config.json` under `peft_model` and fill in the hyperparamters such as `lora_rank`, `lora_alpha` etc., whose content and 
format can be referenced from the corresponding file in Chinese-Alpaca-LoRA.